In [1]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold, KFold
from sklearn import linear_model, metrics, preprocessing
from math import sqrt, floor
from datetime import datetime, timedelta
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input, LSTM,Dense,RepeatVector, TimeDistributed , LeakyReLU
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, IsolationForest
from sklearn.metrics import r2_score, make_scorer, accuracy_score, mean_squared_error, mean_absolute_error
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
import tensorflow as tf
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
import time
from sklearn.pipeline import make_pipeline, Pipeline
import math
from sklearn.decomposition import PCA
from numpy import array
from keras.utils.vis_utils import plot_model
import keras
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input,Conv2D, Conv1D, Conv1DTranspose,MaxPooling1D

import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from keras import optimizers, Sequential
from sklearn.metrics import f1_score
import pickle
import ast


path_errors = 'datapath'
path_features = 'datapath'
df_errors = pd.read_csv(path_errors)
df_features = pd.read_csv(path_features)
total_features = list(df_features[(df_features['True/False'] == 'TRUE') | (df_features['True/False'] == 'NotAvailable')]['Features'])
tot_feat = pd.read_csv('datapath')
tot_feat = list(tot_feat['feat'])

def create_directory(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

def flatten(X):
    flattened_X = np.empty((X.shape[0], X.shape[2]))  # sample x features array.
    for i in range(X.shape[0]):
        flattened_X[i] = X[i, (X.shape[1]-1), :]
    return(flattened_X)


def ConvAEModel(timesteps = 1, n_features=137, layer1=100, layer2=60, layer3=40, encoding_dim=4):
    input = Input(shape=(timesteps, n_features))
    x = Conv1D(filters=layer1, activation="relu",kernel_size=1, padding="same")(input)
    x = Conv1D(filters=layer2,  activation="relu", kernel_size=1,padding="same")(x)
    x = Conv1D(filters=layer3,  activation="relu", kernel_size=1,padding="same")(x)
    x = Conv1D(filters=encoding_dim,  activation="relu",kernel_size=1, padding="same")(x)
    encoder = Model(input,x)
    x = Conv1DTranspose(filters=layer3, activation="relu", kernel_size=1,padding="same")(x)
    x = Conv1DTranspose(filters=layer2, activation="relu", kernel_size=1,padding="same")(x)
    x = Conv1DTranspose(filters=layer1,  activation="relu",kernel_size=1, padding="same")(x)
    x = Conv1DTranspose(filters=n_features,  activation="relu",kernel_size=1, padding="same")(x)
    autoencoder = Model(input, x)
    autoencoder.compile(optimizer="adam", loss="mse")
    return autoencoder

def getHyperParams(file, df_hyp):
    param_dict = ast.literal_eval(df_hyp.loc[df_hyp['Inverter'] == file]['Feature Importance'].values[0])
    print(param_dict)
    return param_dict['model__batch_size'], param_dict['model__encoding_dim'], param_dict['model__epochs'],param_dict['model__layer1'], param_dict['model__layer2'] , param_dict['model__layer3'] 

callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
dayWiseResultsTest = {}
dayWiseResultsTrain = {}
for itr in range(1,4):
    RESULTS_PATH_TRAIN = f'datapath'
    RESULTS_PATH_TEST = f'datapath'
    for dayStart in [14, 30, 90]:
        hyp = '2W' if dayStart == 14 else '1M' if dayStart == 30 else '3M' 
        df_hyp = pd.read_csv(f"datapath/ConvAE{hyp}.csv")
        print(dayStart)
        daysrange = str(dayStart)
        source_dict_results = {}
        source_dict_results_train = {}
        lst=[]
        for i in range(19):
            tot_feat.append('ErrBits')
            print(i)
            inverter = i
            df_inv_0 = pd.read_csv(f'datapath/{inverter}.csv')
            df_inv_0['Timestamp'] = pd.to_datetime(df_inv_0['Timestamp'])
            df_inv_0.sort_values(by='Timestamp', inplace=True)
            df_inv_0 = df_inv_0.set_index(df_inv_0['Timestamp'])

            format = '%Y-%m-%d %H:%M:%S'
            error_date = datetime.strptime(df_errors[(df_errors['Inverter'] == inverter) & (df_errors['had_failure'] == True)]['failure_time'].values[0][0:19], format)
            start_days = 30+dayStart
            start_date = error_date-timedelta(days= start_days)
            end_date = error_date+timedelta(days= 1*30)
            split_date = error_date-timedelta(days= 1*30)
            df_inv_0 = df_inv_0[(df_inv_0.index >= start_date) & (df_inv_0.index < end_date)]
            df_inv_0 = pd.get_dummies(df_inv_0)

            for feat in tot_feat:
                if feat not in df_inv_0.columns:
                    df_inv_0[feat] = 0
            df_inv_0 = df_inv_0[tot_feat]
            df_inv_0 = df_inv_0.dropna()

            df_target = df_inv_0[['ErrBits']]
            tot_feat.remove('ErrBits')
            df_inv_0 = df_inv_0[tot_feat]

            Xtrain_0, Xtest_0 = df_inv_0[(df_inv_0.index >= start_date) & (df_inv_0.index< split_date)], df_inv_0[(df_inv_0.index >= split_date) & (df_inv_0.index< end_date)]
            print(error_date, start_date, split_date, end_date)
            print(df_inv_0.shape, Xtrain_0.shape, Xtest_0.shape)
            Timestamp_train_0, Timestamp_test_0 = df_inv_0[(df_inv_0.index >= start_date) & (df_inv_0.index< split_date)].index, df_inv_0[(df_inv_0.index >= split_date) & (df_inv_0.index< end_date)].index
            ytrain, ytest = df_target[(df_target.index >= start_date) & (df_target.index< split_date)], df_target[(df_target.index >= split_date) & (df_target.index< end_date)]
            print(Xtrain_0.shape)
            if Xtrain_0.shape[0]>1:
                X_scaler = MinMaxScaler()
                X_scaler.fit(Xtrain_0)
                xtrain = X_scaler.transform(Xtrain_0)
                xtest = X_scaler.transform(Xtest_0)
                xtrain = xtrain.reshape(xtrain.shape[0],1, xtrain.shape[1])
                xtest = xtest.reshape(xtest.shape[0],1, xtest.shape[1])
                model__batch_size,model__encoding_dim, model__epochs, model__layer1, model__layer2, model__layer3 = getHyperParams(inverter, df_hyp)
                model  = ConvAEModel(layer1=model__layer1, layer2=model__layer2,layer3=model__layer3, encoding_dim=model__encoding_dim)
                model.fit(xtrain, xtrain, epochs=model__epochs, batch_size=model__batch_size, verbose=False,validation_split=0.1, callbacks=callback1)
                test_enc = model.predict(xtest)
                train_enc = model.predict(xtrain)
                test_mses = np.mean(np.power(flatten(xtest)- flatten(test_enc), 2), axis=1)
                train_mses = np.mean(np.power(flatten(xtrain)- flatten(train_enc), 2), axis=1)
                test_maes = np.mean(flatten(xtest)- flatten(test_enc), axis=1)
                train_maes = np.mean(flatten(xtrain)- flatten(train_enc), axis=1)

                mse_threshold = np.max(train_mses)
                mae_threshold = np.max(train_maes)

                ypred_mse = np.where(test_mses>mse_threshold,1,0)
                ypred_mae = np.where(test_maes>mae_threshold,1,0)


                test_df = pd.DataFrame(ytest)
                test_df['mse_scores'] = test_mses
                test_df['ytrue'] = np.where(test_df['ErrBits']>0, 1, 0)
                test_df['mae_scores'] = test_maes
                test_df['ypred_mse'] = ypred_mse
                test_df['ypred_mae'] = ypred_mae
                lst.append(f1_score(y_true=test_df['ytrue'], y_pred=test_df['ypred_mse']))
                test_df = test_df[['ytrue','ypred_mse','ypred_mae','mse_scores', 'mae_scores']]
                source_dict_results[f'inv_{i}'] = test_df
                
                train_df = pd.DataFrame(ytrain)
                train_df['mse_scores'] = train_mses
                train_df['ytrue'] = np.where(train_df['ErrBits']>0, 1, 0)
                train_df['mae_scores'] = train_maes
                train_df = train_df[['ytrue','mse_scores', 'mae_scores']]
                source_dict_results_train[f'inv_{i}'] = train_df
                
        dayWiseResultsTest[dayStart] = source_dict_results 
        dayWiseResultsTrain[dayStart] = source_dict_results_train
        print(lst)
        with open(f'{RESULTS_PATH_TRAIN}/train_STL_ConvAE.pkl', 'wb') as f:
            pickle.dump(dayWiseResultsTrain, f) 
        with open(f'{RESULTS_PATH_TEST}/test_STL_ConvAE.pkl', 'wb') as f:
            pickle.dump(dayWiseResultsTest, f) 

2023-08-29 21:14:55.727843: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-29 21:14:55.727888: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


14
0
2019-10-11 08:24:20 2019-08-28 08:24:20 2019-09-11 08:24:20 2019-11-10 08:24:20
(14406, 137) (2761, 137) (11645, 137)
(2761, 137)
{'model__batch_size': 128, 'model__encoding_dim': 48, 'model__epochs': 100, 'model__layer1': 128, 'model__layer2': 64, 'model__layer3': 64}


2023-08-29 21:16:11.453202: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-29 21:16:11.461768: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-08-29 21:16:11.469457: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-08-29 21:16:11.477983: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-08-29 21:16:11.537102: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

1
2019-04-02 15:20:48 2019-02-17 15:20:48 2019-03-03 15:20:48 2019-05-02 15:20:48
(14115, 137) (2884, 137) (11231, 137)
(2884, 137)
{'model__batch_size': 256, 'model__encoding_dim': 32, 'model__epochs': 200, 'model__layer1': 64, 'model__layer2': 64, 'model__layer3': 64}
2
2018-12-28 09:46:38 2018-11-14 09:46:38 2018-11-28 09:46:38 2019-01-27 09:46:38
(14142, 137) (2884, 137) (11258, 137)
(2884, 137)
{'model__batch_size': 128, 'model__encoding_dim': 16, 'model__epochs': 100, 'model__layer1': 64, 'model__layer2': 64, 'model__layer3': 32}
3
2018-10-13 16:26:39 2018-08-30 16:26:39 2018-09-13 16:26:39 2018-11-12 16:26:39
(13338, 137) (2199, 137) (11139, 137)
(2199, 137)
{'model__batch_size': 256, 'model__encoding_dim': 48, 'model__epochs': 200, 'model__layer1': 64, 'model__layer2': 32, 'model__layer3': 32}
4
2018-10-13 13:18:07 2018-08-30 13:18:07 2018-09-13 13:18:07 2018-11-12 13:18:07
(13150, 137) (2225, 137) (10925, 137)
(2225, 137)
{'model__batch_size': 128, 'model__encoding_dim': 32, '

Epoch 00156: early stopping
11
2019-08-18 12:36:11 2019-06-19 12:36:11 2019-07-19 12:36:11 2019-09-17 12:36:11
(17928, 137) (6176, 137) (11752, 137)
(6176, 137)
{'model__batch_size': 256, 'model__encoding_dim': 64, 'model__epochs': 100, 'model__layer1': 128, 'model__layer2': 64, 'model__layer3': 64}
12
2019-08-24 13:13:44 2019-06-25 13:13:44 2019-07-25 13:13:44 2019-09-23 13:13:44
(17977, 137) (5741, 137) (12236, 137)
(5741, 137)
{'model__batch_size': 256, 'model__encoding_dim': 48, 'model__epochs': 100, 'model__layer1': 128, 'model__layer2': 32, 'model__layer3': 64}
13
2019-11-05 11:51:23 2019-09-06 11:51:23 2019-10-06 11:51:23 2019-12-05 11:51:23
(18302, 137) (6180, 137) (12122, 137)
(6180, 137)
{'model__batch_size': 256, 'model__encoding_dim': 64, 'model__epochs': 100, 'model__layer1': 128, 'model__layer2': 64, 'model__layer3': 64}
14
2019-10-12 15:00:20 2019-08-13 15:00:20 2019-09-12 15:00:20 2019-11-11 15:00:20
(18498, 137) (6159, 137) (12339, 137)
(6159, 137)
{'model__batch_size'

[0.9897897897897898, 0.20159716060337177, 0.9806157354618017, 0.7455211726384365, 0.5354029850746269, 0.9905033238366572, 0.7891891891891891, 0.6858749121574139, 0.9963846710050615, 0.9389683032939714, 0.7922330097087379, 0.958547910510764, 0.999186330349878, 0.9983038526774898, 0.981698732989207, 0.8857142857142858, 0.8524590163934426, 0.2709163346613546, 0.9686151431934091]
14
0
2019-10-11 08:24:20 2019-08-28 08:24:20 2019-09-11 08:24:20 2019-11-10 08:24:20
(14406, 137) (2761, 137) (11645, 137)
(2761, 137)
{'model__batch_size': 128, 'model__encoding_dim': 48, 'model__epochs': 100, 'model__layer1': 128, 'model__layer2': 64, 'model__layer3': 64}
1
2019-04-02 15:20:48 2019-02-17 15:20:48 2019-03-03 15:20:48 2019-05-02 15:20:48
(14115, 137) (2884, 137) (11231, 137)
(2884, 137)
{'model__batch_size': 256, 'model__encoding_dim': 32, 'model__epochs': 200, 'model__layer1': 64, 'model__layer2': 64, 'model__layer3': 64}
Epoch 00142: early stopping
2
2018-12-28 09:46:38 2018-11-14 09:46:38 2018-

8
2019-09-21 15:53:48 2019-07-23 15:53:48 2019-08-22 15:53:48 2019-10-21 15:53:48
(14781, 137) (6180, 137) (8601, 137)
(6180, 137)
{'model__batch_size': 128, 'model__encoding_dim': 64, 'model__epochs': 100, 'model__layer1': 128, 'model__layer2': 64, 'model__layer3': 32}
9
2020-01-19 07:22:19 2019-11-20 07:22:19 2019-12-20 07:22:19 2020-02-18 07:22:19
(18326, 137) (6180, 137) (12146, 137)
(6180, 137)
{'model__batch_size': 256, 'model__encoding_dim': 64, 'model__epochs': 100, 'model__layer1': 128, 'model__layer2': 32, 'model__layer3': 32}
10
2019-04-10 12:17:11 2019-02-09 12:17:11 2019-03-11 12:17:11 2019-05-10 12:17:11
(18018, 137) (6165, 137) (11853, 137)
(6165, 137)
{'model__batch_size': 256, 'model__encoding_dim': 48, 'model__epochs': 200, 'model__layer1': 128, 'model__layer2': 32, 'model__layer3': 64}
11
2019-08-18 12:36:11 2019-06-19 12:36:11 2019-07-19 12:36:11 2019-09-17 12:36:11
(17928, 137) (6176, 137) (11752, 137)
(6176, 137)
{'model__batch_size': 256, 'model__encoding_dim': 6

17
2019-10-18 07:26:00 2019-06-20 07:26:00 2019-09-18 07:26:00 2019-11-17 07:26:00
(30278, 137) (17939, 137) (12339, 137)
(17939, 137)
{'model__batch_size': 256, 'model__encoding_dim': 32, 'model__epochs': 100, 'model__layer1': 128, 'model__layer2': 64, 'model__layer3': 64}
18
2020-05-23 13:56:28 2020-01-24 13:56:28 2020-04-23 13:56:28 2020-06-22 13:56:28
(30843, 137) (18533, 137) (12310, 137)
(18533, 137)
{'model__batch_size': 128, 'model__encoding_dim': 64, 'model__epochs': 200, 'model__layer1': 128, 'model__layer2': 64, 'model__layer3': 64}
Epoch 00175: early stopping
[0.977461447212337, 0.19603106125970662, 0.9872576177285318, 0.7454452926208651, 0.5356588221239996, 0.9830348727615458, 0.7891891891891891, 0.7103347889374091, 0.9963846710050615, 0.9622929936305733, 0.8869565217391304, 0.9887620872898335, 0.9995929995929995, 0.9983038526774898, 0.9957163255592575, 0.8611111111111112, 0.896551724137931, 0.5950413223140496, 0.999797529864345]
14
0
2019-10-11 08:24:20 2019-08-28 08:24:2

7
2019-05-27 04:57:13 2019-03-28 04:57:13 2019-04-27 04:57:13 2019-06-26 04:57:13
(18499, 137) (6180, 137) (12319, 137)
(6180, 137)
{'model__batch_size': 128, 'model__encoding_dim': 32, 'model__epochs': 100, 'model__layer1': 64, 'model__layer2': 64, 'model__layer3': 64}
8
2019-09-21 15:53:48 2019-07-23 15:53:48 2019-08-22 15:53:48 2019-10-21 15:53:48
(14781, 137) (6180, 137) (8601, 137)
(6180, 137)
{'model__batch_size': 128, 'model__encoding_dim': 64, 'model__epochs': 100, 'model__layer1': 128, 'model__layer2': 64, 'model__layer3': 32}
9
2020-01-19 07:22:19 2019-11-20 07:22:19 2019-12-20 07:22:19 2020-02-18 07:22:19
(18326, 137) (6180, 137) (12146, 137)
(6180, 137)
{'model__batch_size': 256, 'model__encoding_dim': 64, 'model__epochs': 100, 'model__layer1': 128, 'model__layer2': 32, 'model__layer3': 32}
10
2019-04-10 12:17:11 2019-02-09 12:17:11 2019-03-11 12:17:11 2019-05-10 12:17:11
(18018, 137) (6165, 137) (11853, 137)
(6165, 137)
{'model__batch_size': 256, 'model__encoding_dim': 48,

Epoch 00113: early stopping
16
2019-10-26 08:15:14 2019-06-28 08:15:14 2019-09-26 08:15:14 2019-11-25 08:15:14
(30765, 137) (18425, 137) (12340, 137)
(18425, 137)
{'model__batch_size': 128, 'model__encoding_dim': 32, 'model__epochs': 100, 'model__layer1': 64, 'model__layer2': 64, 'model__layer3': 64}
17
2019-10-18 07:26:00 2019-06-20 07:26:00 2019-09-18 07:26:00 2019-11-17 07:26:00
(30278, 137) (17939, 137) (12339, 137)
(17939, 137)
{'model__batch_size': 256, 'model__encoding_dim': 32, 'model__epochs': 100, 'model__layer1': 128, 'model__layer2': 64, 'model__layer3': 64}
Epoch 00099: early stopping
18
2020-05-23 13:56:28 2020-01-24 13:56:28 2020-04-23 13:56:28 2020-06-22 13:56:28
(30843, 137) (18533, 137) (12310, 137)
(18533, 137)
{'model__batch_size': 128, 'model__encoding_dim': 64, 'model__epochs': 200, 'model__layer1': 128, 'model__layer2': 64, 'model__layer3': 64}
Epoch 00109: early stopping
[0.977461447212337, 0.20159716060337177, 0.9911012235817576, 0.7318876786249625, 0.535658822